# 決定木のアルゴリズム

<img src='desitiontree.png'>

## 目的関数

### ジニ不純度

$$
I_G(t) = 1 - \sum_{i=1}^c p(i|t)^2
$$

これを目的関数にとる。

これが大きいほどそのノード内にいろんなものがごちゃごちゃしていて、小さいほどそのノード内が整理されていることを示す。

### 情報利得

$$
IG(D_p, f) = I_G(D_p) - \frac{N_{left}}{N_p} I_G(D_{left}) - \frac{N_{right}}{N_p} I_G(D_{right}) 
$$

$
親ノード D_p => 子ノード D_{left}, D_{right} と分ける\\
親ノード内のトレーニングサンプルの数: N_p\\
子ノード内のトレーニングサンプルの数をそれぞれN_{left}、N_{right}\\
分割する特徴量: f
$

つまり　**親の不純度 - 子の不純度** ってこと！

これが大きい質問が**良い質問**、ということになる。

過学習を起こしやすいので、ノードの深さの上限を指定したり、ノード分けでだんだん小さくなっていく (情報利得) × (ノード内のデータ数割合) に閾値を設けたりする。

また、他のモデルとして、ランダムフォレストや勾配ブースティング木などのアンサンブルも存在する（今度できたら？）

### 説明変数の重要度

決定木がよく使われる理由の一つに説明変数の重要度を出力できる、と言うことがある。

情報利得が最大になるようにで分割するので、全情報利得のうちの、その説明変数が占める割合で表される。

$$
FI(F_j) = \sum_{t \in N_{f_j}}IG(t,j_j)n_t
$$

$
j番目の特徴量f_jで分割されたノードの集合:n_{f_j}\\
N_{f_j}に含まれるノードtのサンプル数: n_t\\
f_jの重要度：FI(f_j)
$

正規化された特徴量は、


$$
FI_n(f_j) = \frac{FI_(f_j)}{\sum_{j=1}^{n} FI(f_j)}
$$

と表される

## 具体的なアルゴリズム

1. 特徴量の１つに注目し、その値を昇順に並べ、各隣り合う値の中間値で分けて行き、情報利得を参照していく
2. その中で情報利得が最大になる分け方を選択(もし同じになればランダム)
3. これを繰り返していく

# 実装

## 方針

機能は以下の通り

1. fit(data, target)で学習
2. predict(data)で予測
3. score(data, target)で正答率
4. feature\_importances\_で重要度を出す

再帰関数で書いていく

<font size=1px>（オブジェクト指向っぽくかけたらいいな。）</font>

_NodeクラスとDecisionTreeクラスと_TreeAnalysiusの3つのクラスからなる。

決定木自体の過学習回避には、(情報利得) × (ノード内のデータ数割合)　の閾値設定とmax_depthの二つがあるが、今回は両方実装する!

## 実装！

In [1]:
import numpy as np

In [2]:
class _Node:
        #data, target は np.array
        def __init__(self, data, max_depth, random_state, standard):
            self.right = None
            self.left = None
            self.max_depth = max_depth
            self.random_state = random_state
            self.n_features = data.shape[1]
            self.samples = data.shape[0]
            self.label = None
            self.standard = None
            self.best_feature_idx = None
            self.best_threshold = None
        # 子ノード(left, right)に分ける
        def split(self, data, target, depth):
            self.depth = depth
            
            #自ノードの分析
            class_cnt = {i: len(target[target==i]) for i in np.unique(target)}
            self.label = max((value, key) for (key, value) in class_cnt.items())[1]
            
            if depth == self.max_depth:
                return
            self.gini_idx = self.gini(target)
            
            #各情報利得を計算し、最大のものを記憶
            self.best_info_gain = 0
            
            if self.random_state is not None:
                np.random.seed(self.random_state)
            loop_order = np.random.permutation(self.n_features)
            for i in loop_order:
                unique_features = np.unique(data[:, i])
                self.thresholds = (unique_features[:-1] + unique_features[1:]) / 2
                for threshold in self.thresholds:
                    child_left = target[data[:, i] < threshold]
                    child_right = target[data[:, i] >= threshold]
                    info_gain = self.info_gain(child_left, child_right, target)
                    
                    if info_gain > self.best_info_gain:
                        self.best_info_gain = info_gain
                        self.best_feature_idx = i
                        self.best_threshold = threshold
                        self.child_left = child_left
                        self.child_right = child_right
                        
            if not(self.best_info_gain):
                return
            # 再帰させる
            data_left = data[data[:, self.best_feature_idx] < self.best_threshold]
            data_right = data[data[:, self.best_feature_idx] >= self.best_threshold]
            
            target_left = target[data[:, self.best_feature_idx] < self.best_threshold]
            target_right = target[data[:, self.best_feature_idx] >= self.best_threshold]
            
            self.left = _Node(data_left, self.max_depth, self.random_state, self.standard)
            self.right = _Node(data_right, self.max_depth, self.random_state, self.standard)
            
            self.left.split(data_left, target_left, self.depth+1)
            self.right.split(data_right, target_right, self.depth+1)
            
        def predict(self, data):
            if not(self.best_info_gain):
                return self.label
            if data[self.best_feature_idx] < self.best_threshold:
                return self.left.predict(data)
            return self.right.predict(data)
            
        def gini(self, target):
            sigma = 0
            for i in np.unique(target):
                sigma += (len(target[target==i]) / len(target)) ** 2
            return (1 - sigma)

        def info_gain(self, left, right, target):
            return (self.gini_idx - len(left) / len(target) * self.gini(left) - len(right) / len(target) * self.gini(right))
        
        def feature_importances(self):
            self.feature_importances_ = np.zeros(self.n_features)
            self.feature_importances_[self.best_feature_idx] += self.best_info_gain * self.samples
            if not(self.best_info_gain):
                return self.feature_importances_
            return (self.feature_importances_ + self.left.feature_importances() + self.right.feature_importances())
        def trim(self, standard):
            pass

In [3]:
class MyDecisionTree:
    def __init__(self, max_depth=None, random_state=None, standard=None):
        self.max_depth = max_depth
        self.random_state = random_state
        self.standard = standard
        
    def fit(self, data, target):
        self.root = _Node(data, self.max_depth, self.random_state, self.standard)
        self.root.split(data, target, 0)
        self.root.trim(self.standard)
        self.feature_importances_ = self.feature_importances()
        
    def  predict(self, data):
        ans = []
        if len(data.shape) == 1:
            return self.root.predict(data)
        for d in data:
            ans.append(self.root.predict(d))
        return np.array(ans)
    
    def score(self, data, target):
        pred = self.predict(data)
        return(len(pred[pred==target]) / len(target))
    
    def feature_importances(self):
        pure_feature_importances_ = self.root.feature_importances()
        return (pure_feature_importances_ / sum(pure_feature_importances_))

# 実際に使ってみる

sklearnのDecisionTreeClassifierと比較する。

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## みんな大好きアヤメ

In [5]:
from sklearn.datasets import load_iris
iris = load_iris()

In [6]:
X = iris.data
y = iris.target

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
mdt = MyDecisionTree(random_state=0)

In [9]:
mdt.fit(X_train, y_train)

In [10]:
print(mdt.score(X_train,y_train))
print(mdt.score(X_test, y_test))

1.0
0.9777777777777777


In [11]:
mdt.feature_importances_

array([0.        , 0.04300928, 0.90006666, 0.05692405])

In [12]:
dtc = DecisionTreeClassifier(random_state=0)

In [13]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [14]:
print(dtc.score(X_train,y_train))
print(dtc.score(X_test, y_test))

1.0
0.9777777777777777


In [15]:
dtc.feature_importances_

array([0.        , 0.02150464, 0.39766951, 0.58082584])

なかなかの精度！？

## kaggleの乳がんのデータセット

In [16]:
import pandas as pd
brest_cancer = pd.read_csv('data.csv')

In [17]:
brest_cancer.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [18]:
brest_cancer = brest_cancer.drop('Unnamed: 32', axis=1)

In [19]:
brest_cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [20]:
X_1 = brest_cancer.iloc[:, 2:]
y_1 = brest_cancer['diagnosis']

In [21]:
X_1.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [22]:
y_1

0      M
1      M
2      M
3      M
4      M
5      M
6      M
7      M
8      M
9      M
10     M
11     M
12     M
13     M
14     M
15     M
16     M
17     M
18     M
19     B
20     B
21     B
22     M
23     M
24     M
25     M
26     M
27     M
28     M
29     M
      ..
539    B
540    B
541    B
542    B
543    B
544    B
545    B
546    B
547    B
548    B
549    B
550    B
551    B
552    B
553    B
554    B
555    B
556    B
557    B
558    B
559    B
560    B
561    B
562    M
563    M
564    M
565    M
566    M
567    M
568    B
Name: diagnosis, Length: 569, dtype: object

In [23]:
mapping = {'B': 0, 'M': 1} #1が陽性
y_1 = y_1.map(mapping)

In [24]:
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.2)

In [25]:
mdt_1 = MyDecisionTree(random_state=0)
dtc_1 = DecisionTreeClassifier(random_state=0)

In [26]:
X_1_train = X_1_train.values
X_1_test = X_1_test.values

In [28]:
%time mdt_1.fit(X_1_train, y_1_train)
%time dtc_1.fit(X_1_train, y_1_train)

CPU times: user 4min 19s, sys: 3.24 s, total: 4min 22s
Wall time: 4min 47s
CPU times: user 10.5 ms, sys: 6.15 ms, total: 16.6 ms
Wall time: 16.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [29]:
print(mdt_1.score(X_1_train, y_1_train))
print(mdt_1.score(X_1_test, y_1_test))

1.0
0.956140350877193


In [30]:
print(dtc_1.score(X_1_train, y_1_train))
print(dtc_1.score(X_1_test, y_1_test))

1.0
0.9649122807017544


In [31]:
print(mdt_1.feature_importances_)
print(dtc_1.feature_importances_)

[0.         0.         0.0161772  0.         0.02752371 0.
 0.         0.         0.         0.         0.00181945 0.00788639
 0.         0.01287888 0.00146393 0.         0.         0.
 0.00838818 0.         0.08007484 0.04835935 0.         0.03296776
 0.00629113 0.         0.01709889 0.72239902 0.01667127 0.        ]
[0.         0.00786392 0.         0.         0.02752371 0.
 0.         0.         0.         0.         0.00936881 0.00788639
 0.00923497 0.0037749  0.00775506 0.         0.         0.
 0.01546571 0.         0.08007484 0.04835935 0.00909968 0.03296776
 0.         0.         0.00910398 0.72239902 0.         0.00912191]


こちらも精度でたが、sklearnに比べて非常に遅い。。。(4分47秒w)

# 課題、感想等

そもそも仕組みが単純なモデルだったので実装することができた。

ただ、計算効率が悪いようだった。

実際はジニ係数の他に、引数によって情報エントロピーも取れるので、その実装や、木の視覚化もできればよかった。

ランダムフォレストの実装もできたらいいな、、、と思う。

# 参考文献

http://darden.hatenablog.com/entry/2016/12/09/221630

http://darden.hatenablog.com/entry/2016/12/15/222447

http://codecrafthouse.jp/p/2014/09/decision-tree/